El entrenamiento y evaluación de los modelos de Machine Learning se realiza en Google Colab con un subconjunto del dataset.

In [1]:
# Instalar e importar librerías útiles
!pip install pyspark
!pip install pandas

import pandas as pd
import os
from google.colab import drive

# Montar Google Drive en Colab (opcional)
drive.mount('/content/drive')

# Elegir la ruta en la que se almacenan los datos
path = "drive/MyDrive/Ciclos/2023-1/Big Data/ProyectoBD/Big Data/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Obtención de datos (opcional)

Debido a que Colab no cuenta con los recursos para acceder y procesar el dataset completo, el siguiente script recopila desde GitHub un subconjunto de este conformado para una cantidad de días variable dentro del primer mes del año, por defecto solo se extraen los datos del primer día, es decir, solo se cuenta con información de tweets publicados el 01/01/2022. Si ya ejecutaste este script o cuentas con datos de otro medio, puedes saltar esta sección

In [ ]:
parte_max = 23 # Número máximo de particiones existentes en el dataset para los datos de un día, se define para obtener todos los datos de una cantidad de días específica (no modificar)
dia_max = 1 # Número de días sobre los cuáles se desea extraer los datos por cada tabla (variable)

# Constantes útiles para acceder a los archivos de las tablas
parte = "00"
dia = "01"

# Lista de tablas del repositorio
tables = ["Summary_Details", "Summary_Hashtag", "Summary_Mentions", "Summary_NER", "Summary_NER_ES", "Summary_Sentiment", "Summary_Sentiment_ES"]

# Extraer archivos csv del github, renombrarlos y convertirlos en formato parquet (incluye pasos de preprocesamiento vistos anteriormente)
for table in tables:
  for d in range(1, dia_max+1):
    for p in range(0, parte_max+1):
      try:
        if d < 10:
          dia = "0"+str(d)
        else:
          dia = str(d)
        if p < 10:
          parte = "0"+str(p)
        else:
          parte = str(p)

        if table == "Summary_NER_ES":
          file = "2022_01_" + dia + "_" + parte + "_Summary_ES_NER.csv"
        else:
          file = "2022_01_" + dia + "_" + parte + "_" + table + ".csv"

        df = pd.read_csv("https://raw.githubusercontent.com/lopezbec/COVID19_Tweets_Dataset/main/" + table + "/2022_01/" + file, dtype={'Tweet_ID': str})

        if 'Country' in df:
          df = df.drop('Country', axis=1)
        df = df.dropna()

        for col in df.columns:
          df = df.rename(columns={col: col.replace(" ", "_")})

        if not os.path.exists(path + "COVID19_Tweets_Dataset_Pq/" + table):
          os.mkdir(path + "COVID19_Tweets_Dataset_Pq/" + table)

        df.to_parquet(
          path=path+"COVID19_Tweets_Dataset_Pq/" + table + "/" + f'{file[:-4]}.parquet',
          engine='pyarrow',
          compression='gzip',
        )
      except:
        continue

# Entrenamiento de modelos

Para realizar las predicciones entrenamos y evaluamos dos modelos de Machine Learning: Random Forest y Regresión Lineal.

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, RegressionEvaluator

# Iniciar sesión en spark
spark = SparkSession.builder \
       .master("local[2]") \
       .appName("test") \
       .config("spark.driver.memory", "6g")\
       .config("spark.executor.memory", "6g")\
       .getOrCreate()

In [3]:
# Leer los archivos parquet
summary_details_df = spark.read.parquet(path + "COVID19_Tweets_Dataset_Pq/Summary_Details")
summary_hashtag_df = spark.read.parquet(path + "COVID19_Tweets_Dataset_Pq/Summary_Hashtag")
summary_mentions_df = spark.read.parquet(path + "COVID19_Tweets_Dataset_Pq/Summary_Mentions")
summary_sentiment_df = spark.read.parquet(path + "COVID19_Tweets_Dataset_Pq/Summary_Sentiment")
summary_ner_df = spark.read.parquet(path + "COVID19_Tweets_Dataset_Pq/Summary_NER")
summary_sentiment_es_df = spark.read.parquet(path + "COVID19_Tweets_Dataset_Pq/Summary_Sentiment_ES")
summary_ner_es_df = spark.read.parquet(path + "COVID19_Tweets_Dataset_Pq/Summary_NER_ES")

In [4]:
# Realizar join de los dataframes en parquet (modificar en base a las columnas que se quiera usar)
joined_df = summary_details_df.select("Tweet_ID", "Likes", "Retweets") \
    .join(summary_sentiment_df.select("Tweet_ID", "Sentiment_Label"), "Tweet_ID") \
    .join(summary_ner_df.select("Tweet_ID", "NER_text", "NER_Label"), "Tweet_ID")

# Extraer una cantidad de filas del dataframe (modificar en base a la cantidad con que se quiera trabajar)
joined_df = joined_df.limit(10000)

In [5]:
# Convertir las etiquetas de sentimiento a valores numéricos
label_indexer = StringIndexer(inputCol="Sentiment_Label", outputCol="label")

# Convertir las columnas categóricas en índices numéricos
indexers = [StringIndexer(inputCol=col, outputCol=col+"_index").setHandleInvalid("keep") for col in ["NER_text", "NER_Label"]]

# Codificar los índices numéricos en representaciones one-hot
encoders = [OneHotEncoder(inputCol=col+"_index", outputCol=col+"_encoded") for col in ["NER_text", "NER_Label"]]

# Seleccionar las características relevantes y combinarlas en una sola columna
feature_cols = ["NER_text_encoded", "Likes", "Retweets", "NER_Label_encoded"]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

In [6]:
# Crear el pipeline de transformación de datos
pipeline = Pipeline(stages=indexers + encoders + [assembler, label_indexer])

# Aplicar el pipeline para transformar los datos
transformed_data = pipeline.fit(joined_df).transform(joined_df)

# Dividir los datos en conjuntos de entrenamiento y prueba
(training_data, testing_data) = transformed_data.randomSplit([0.7, 0.3], seed=138)

In [7]:
# Crear una instancia del clasificador RandomForest
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=10)

# Crear el modelo de Regresión Logística
lr = LogisticRegression(labelCol="label", featuresCol="features")

# Entrenar los modelos con los datos de entrenamiento
modelRF = rf.fit(training_data)
modelLR = lr.fit(training_data)

# Evaluación de modelos

In [8]:
%%timeit
# Realizar predicciones en el conjunto de prueba con el modelo Random Forest
predictionsRF = modelRF.transform(testing_data)

58.5 ms ± 9.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%%timeit
# Realizar predicciones en el conjunto de prueba con el modelo de Regresión Lineal
predictionsLR = modelLR.transform(testing_data)

70.5 ms ± 14.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
# Mostrar algunas predicciones
predictionsRF = modelRF.transform(testing_data)
predictionsLR = modelLR.transform(testing_data)
predictionsRF.select("Sentiment_Label", "prediction", "probability").show(10)
predictionsLR.select("Sentiment_Label", "prediction", "probability").show(10)

+---------------+----------+--------------------+
|Sentiment_Label|prediction|         probability|
+---------------+----------+--------------------+
|        neutral|       1.0|[0.45004128162225...|
|        neutral|       1.0|[0.45004128162225...|
|        neutral|       1.0|[0.45004128162225...|
|        neutral|       1.0|[0.45004128162225...|
|       negative|       1.0|[0.45004128162225...|
|       negative|       1.0|[0.45004128162225...|
|       positive|       1.0|[0.43014753646274...|
|       positive|       1.0|[0.43014753646274...|
|        neutral|       1.0|[0.45004128162225...|
|        neutral|       1.0|[0.45004128162225...|
+---------------+----------+--------------------+
only showing top 10 rows

+---------------+----------+--------------------+
|Sentiment_Label|prediction|         probability|
+---------------+----------+--------------------+
|        neutral|       0.0|[0.45772565187304...|
|        neutral|       0.0|[0.43585955343369...|
|        neutral|       

In [11]:
# Evaluar el rendimiento del modelo en el conjunto de prueba
evaluatorRF = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
evaluatorLR = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")

accuracyRF = evaluatorRF.evaluate(predictionsRF)
accuracyLR = evaluatorLR.evaluate(predictionsLR)
print("Random Forest Accuracy:", accuracyRF)
print("Logistic Regression Accuracy:", accuracyLR)

Random Forest Accuracy: 0.49274406332453824
Logistic Regression Accuracy: 0.7040684633296742
